In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

user = "aacuser1"
pw = "cs340"
host = 'nv-desktop-serviceds.apporto.com'
port = 30017
db = 'aac'
col = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(use, pw, host, port, db, col)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvare.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date

 # Water Rescue filter
filter_options1 = [
    {'label': 'Water Rescue', 'value': 'Water Rescue'},
]
        
# Mountain and Wilderness filter
filter_options2 = [
    {'label': 'Mountain', 'value': 'Mountain'},
    {'label': 'Wilderness', 'value': 'Wilderness'},
]
        
# Disaster Rescue and Individual Tracking filter
filter_options3 = [
    {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
    {'label': 'Individual Tracking', 'value': 'Individual Tracking'}
]


app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        # Water rescue filter
        dcc.RadioItems(
            id='filter-radio',
            options=filter_options1,
            value=[],
        ),
        # Mountain and wilderness filter
        dcc.Dropdown(
            id='filter-dropdown1',
            options=filter_options2,
            multi=True,     # Allow multiple selections
            placeholder='Select rescue types',
        ),
        # Disaster and Individual filter
        dcc.Dropdown(
            id='filter-dropdown2',
            options=filter_options3,
            multi=True,     # Allow multiple selections
            placeholder='Select rescue types',
            ),
        # Reset Button
        html.Button('Reset Filters', id='reset-button', n_clicks=0),
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         sort_action="native",   # Enable native sorting
                         sort_mode="multi",      # Allow multi-column sorting
                         filter_action="native",    # Enable native filtering
                         page_size=10,    # Set the number of rows per page
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-radio', 'value'),
               Input('filter-dropdown1', 'value'),
               Input('filter-dropdown2', 'value')])
def update_dashboard(filter_radio, filter_dropdown1, filter_dropdown2):
    filtered_df = df
    
    if filter_radio == 'Water Rescue':
        filtered_df = df[df['rescue_type'] == 'Water Rescue']
    elif filter_radio == 'Mountain and Wilderness Rescue':
        filtered_df = df[df['rescue_type'].isin(['Mountain', 'Wilderness'])]
    elif filter_radio == 'Disaster Rescue and Individual Tracking':
        filtered_df = df[df['rescue_type'].isin(['Disaster Rescue', 'Individual Tracking'])]
    
    if filter_dropdown1:
        filtered_df = filtered_df[filtered_df['rescue_type'].isin(filter_dropdown1)]
    
    if filter_dropdown2:
        filtered_df = filtered_df[filtered_df['rescue_type'].isin(filter_dropdown2)]
        
    data = filtered_df.to_dict('records')
    return data

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return [html.P("No data available")]      # Return if no data is available
    
    # Convert the data from viewData into a DataFrame
    df_view = pd.DataFrame.from_dict(viewData)
    
    # Create a pie chart using Plotly Express
    fig = px.pie(df_view, names='rescue_type', title='Rescue Type Distribution')
    
    # Return the pie chart as a dcc.Graph component
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# Callback to reset filters
@app.callback(
    [Output('filter-radio', 'value'),
     Output('filter-dropdown1', 'value'),
     Output('filter-dropdown2', 'value')],
    [Input('reset-button', 'n_clicks')]
)
def reset_filters(n_clicks):
    if n_clicks is None:
        return dash.no_update  # No update if reset button not clicked
    
    # Set the value of the filters to empty lists to reset them
    return [], [], []

app.run_server(debug=True)


ModuleNotFoundError: No module named 'animal_shelter'